In [1]:
# IMPORTING HEADER FILES

import numpy as np 
import pandas as pd 
import os
import string
from string import digits
import matplotlib.pyplot as plt
import re
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
# LOADING THE DATA
lines=pd.read_csv("Corpus.csv", encoding='utf-8')
print(lines)
lines=lines[lines['source']=='ted']
print(lines)

           source                                   english_sentence  \
0             ted  politicians do not have permission to do what ...   
1             ted         I'd like to tell you about one such child,   
2       indic2012  This percentage is even greater than the perce...   
3             ted  what we really mean is that they're bad at not...   
4       indic2012  .The ending portion of these Vedas is called U...   
...           ...                                                ...   
127602  indic2012  Examples of art deco construction can be found...   
127603        ted                          and put it in our cheeks.   
127604      tides  As for the other derivatives of sulphur , the ...   
127605      tides  its complicated functioning is defined thus in...   
127606        ted  They've just won four government contracts to ...   

                                           hindi_sentence  
0       राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...  
1       मई आपको

In [3]:
lines=lines[~pd.isnull(lines['english_sentence'])]       # Removal of null values
lines.drop_duplicates(inplace=True)                      # Dropping duplicate lines
# SIMPLE RANDOM SAMPLING FROM DATA
lines=lines.sample(n=25000,random_state=42)           
lines.shape

(25000, 3)

In [4]:
# TEXT PREPROCESSING(TOKEN NORMALIZATION)

# Conversion of text to lower case
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())   
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

# Removal of quotation marks from text
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

# Removal of punctuations from text
exclude = set(string.punctuation)      # Set containing all punctuations
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

# Removal of numerical digits from text
remove_digits = str.maketrans('', '', digits)     # Function mapping all digits to empty string
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))   # Application of the mapping
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))  # Removal of regional language digits

# Removal of extra spaces from text
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [5]:
# Addition of START_ and _END to the output sequence for ease of LSTM training 
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')   

In [6]:
# CREATION OF HINDI AND ENGLISH VOCABULARIES

all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

print(all_eng_words,all_hindi_words)

# Adding length factor in the list
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

# Removal of long sentences to avoid complexity while training
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))    # Sorting words in the vocabularies
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)       # Storing number of words in each vocabulary

with open('eng_vocab.txt', 'w', encoding='utf-8') as filehandle:
    for listitem in all_eng_words:
        filehandle.write(listitem+'\n')
        
with open('hin_vocab.txt', 'w', encoding='utf-8') as filehandle:
    for listitem in all_hindi_words:
        filehandle.write(listitem+'\n')

{'expanding', 'replied', 'perspire', 'ministry', 'biomimetic', 'gopnik', 'notation', 'alleviated', 'pamela', 'jarshaped', 'handling', 'depends', 'v', 'seniority', 'generation', 'hoop', 'blogged', 'zeitgeist', 'hangers', 'walker', 'pick', 'postretirement', 'sentimental', 'fashion', 'thirdsiders', '“achilles', 'senegalese', 'chris', 'projustice', 'diversity', 'falls', 'spiked', 'analogous', 'america', '“eating', 'linear', 'hostage', 'privatization', 'shoulders', 'suck”', 'hips”', 'dc', 'swami', 'convict', 'gatekeepers', 'divide', 'radar', 'needlepoint', 'material', '“oops', 'identities', 'जापानी', 'sensitivity', 'ides', 'racket', 'sorghum', 'twostate', 'blackberry', 'star', 'medicine', 'grandmother', 'galaxys', 'conferences', 'audiences', 'bhārata', 'invited', 'secondary', '“design', 'marley', 'piracy”', 'nirvana', 'consolidated', 'lane', 'pollinate', 'agreement', 'arithmetic', 'hacked', 'transformational', 'solace', 'beth', 'sparking', 'dodging', 'irregular', 'lava', 'well”', 'moodenhan

In [7]:
# Indexing the vocabulary
num_decoder_tokens += 1  #for zero padding
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])   # Index on RHS and word on LHS

# Rearranging index to LHS
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

lines = shuffle(lines)  # Shuffling the order of lines in the 

In [8]:
# SPLITTING DATASET INTO TRAIN, TEST AND SPLIT
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)


In [9]:
# FUNCTION FOR GENERATION OF BATCHES AND ARITHMETIC MATRICES FOR EASE OF TRAINING
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')  # Array for storing input text
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')  # Array for storing output text
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')  # Array for storing corresponding one hot encoded outputs
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):  # i:index; input_text:eng; output_text:hin
                for t, word in enumerate(input_text.split()):       
                    encoder_input_data[i, t] = input_token_index[word] # Entering word indexes(in vocab) of input text into array
                
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:    # To remove the END_ token from consideration
                        decoder_input_data[i, t] = target_token_index[word] # Entering word indexes(in vocab) of output text into array

                    if t>0:           # To remove the START_ token from consideration
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1. # Storing target words as one hot encoded vectors where i:index of sentence; t:index of word; third attribute:corresponding index in vector where 1 is to be placed, rest all r made 0    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)  # 'yield' used to return all three matrices

In [10]:
# CONSTRUCTING MODEL ARCHITECTURE
latent_dim = 300
encoder_inputs = Input(shape=(None,))      # Input Layer

# Embedding Layer - Creates embedding matrix of shape(num_encoder_tokens,dim) and embeds values that are present in encoder_inputs
# Thus an embedding vector of 'dim' size is created for each of the 'num_encoder_tokens' present in the vocabulary
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)   

# LSTM Layer - Shape defined as 'dim' as the embedding vector of size 'dim' is passed for every word that is being analyzed
encoder_lstm = LSTM(latent_dim, return_state=True) 

# Storing the LSTM outputs ie the output, hidden state and cell state in variables for comparison
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb) 

# The 'encoder_outputs' discarded and only the states are kept
encoder_states = [state_h, state_c]

In [11]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [16]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    4209000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    5262300     input_2[0][0]                    
____________________________________________________________________________________________

C:\Users\Others\Anaconda3\envs\tp\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
154/154 [==============================] - 79s 516ms/step - loss: 0.0332 - val_loss: 3.7579
Epoch 2/100
154/154 [==============================] - 79s 515ms/step - loss: 0.0272 - val_loss: 3.6500
Epoch 3/100
154/154 [==============================] - 79s 512ms/step - loss: 0.0253 - val_loss: 3.7334
Epoch 4/100
154/154 [==============================] - 79s 512ms/step - loss: 0.0239 - val_loss: 3.5787
Epoch 5/100
154/154 [==============================] - 76s 494ms/step - loss: 0.0229 - val_loss: 3.7885
Epoch 6/100
154/154 [==============================] - 79s 515ms/step - loss: 0.0217 - val_loss: 3.8430
Epoch 7/100
154/154 [==============================] - 82s 529ms/step - loss: 0.0210 - val_loss: 3.4922
Epoch 8/100
154/154 [==============================] - 78s 505ms/step - loss: 0.0200 - val_loss: 3.6451
Epoch 9/100
154/154 [==============================] - 79s 515ms/step - loss: 0.0195 - val_loss: 3.6404
Epoch 10/100
154/154 [==============================] - 78s 503m

Epoch 79/100
154/154 [==============================] - 79s 514ms/step - loss: 0.0058 - val_loss: 4.2558
Epoch 80/100
154/154 [==============================] - 82s 531ms/step - loss: 0.0057 - val_loss: 4.1435
Epoch 81/100
154/154 [==============================] - 83s 536ms/step - loss: 0.0058 - val_loss: 4.1049
Epoch 82/100
154/154 [==============================] - 82s 535ms/step - loss: 0.0056 - val_loss: 3.9985
Epoch 83/100
154/154 [==============================] - 81s 523ms/step - loss: 0.0057 - val_loss: 4.2062
Epoch 84/100
154/154 [==============================] - 81s 526ms/step - loss: 0.0054 - val_loss: 4.2316
Epoch 85/100
154/154 [==============================] - 83s 539ms/step - loss: 0.0056 - val_loss: 3.8826
Epoch 86/100
154/154 [==============================] - 78s 508ms/step - loss: 0.0055 - val_loss: 4.0148
Epoch 87/100
154/154 [==============================] - 87s 562ms/step - loss: 0.0055 - val_loss: 3.9741
Epoch 88/100
154/154 [==============================] -

In [13]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [14]:
def decode_sequence(input_seq):
    
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [15]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: to be able to communicate
Actual Hindi Translation:  हम बातचीते करें 
Predicted Hindi Translation:  हम बातचीते करें 
Input English sentence: where the volunteers supplement a highly skilled career staff
Actual Hindi Translation:  जहाँ स्वयंसेवक अत्यधिक कुशल अग्निशमन कर्मियों की सहायता करते हैं 
Predicted Hindi Translation:  जहाँ स्वयंसेवक अत्यधिक कुशल अग्निशमन कर्मियों की सह
Input English sentence: the house looks beautiful you hang them in the window
Actual Hindi Translation:  घर सुंदर दिखता है अगर आप उन्हें खिडकियों में लगा दें। 
Predicted Hindi Translation:  घर सुंदर और आप उनसे बस देख रहा था 
